In [1]:
import numpy as np
import matplotlib.pyplot as plt

import zfit
from zfit.core.loss import ExtendedUnbinnedNLL
from zfit.minimize import Minuit

from skstats.hypotests.calculators import AsymptoticCalculator
from skstats.hypotests import Discovery
from skstats.hypotests.parameters import POI

/Users/matthieumarinangeli/anaconda3/envs/tfn/lib/python3.7/site-packages/zfit/util/execution.py:61: UserWarning: Not running on Linux. Determining available cpus for thread can failand be overestimated. Workaround (only if too many cpus are used):`zfit.run.set_n_cpu(your_cpu_number)`
  warnings.warn("Not running on Linux. Determining available cpus for thread can fail"


ModuleNotFoundError: No module named 'core'

### Fit of a Gaussian signal over an exponential background:

In [ ]:
bounds = (0.1, 3.0)

# Data and signal

np.random.seed(0)
tau = -2.0
beta = -1/tau
data = np.random.exponential(beta, 300)
peak = np.random.normal(1.2, 0.1, 25)
data = np.concatenate((data,peak))
data = data[(data > bounds[0]) & (data < bounds[1])]

plt.hist(data, bins=100, histtype='step');

In [ ]:
obs = zfit.Space('x', limits=bounds)

In [ ]:
lambda_ = zfit.Parameter("lambda",-2.0, -4.0, -1.0)
Nsig = zfit.Parameter("Nsig", 20., -20., len(data))
Nbkg = zfit.Parameter("Nbkg", len(data), 0., len(data)*1.1)

In [ ]:
signal = Nsig * zfit.pdf.Gauss(obs=obs, mu=1.2, sigma=0.1)
background =  Nbkg * zfit.pdf.Exponential(obs=obs, lambda_=lambda_)
tot_model = signal + background

In [ ]:
# Create the negative log likelihood
from zfit.core.loss import ExtendedUnbinnedNLL, UnbinnedNLL
data_ = zfit.data.Data.from_numpy(obs=obs, array=data)
nll = ExtendedUnbinnedNLL(model=[tot_model], data=[data_], fit_range=[obs]) 

In [ ]:
# Instantiate a minuit minimizer
minimizer = Minuit()

In [ ]:
# minimisation of the loss function
minimum = minimizer.minimize(loss=nll)

In [ ]:
def plotfitresult(pdf, bounds, nbins, data):
    x = np.linspace(*bounds, num=1000)
    pdf = zfit.run(tot_model.pdf(x, norm_range=bounds)* tot_model.get_yield())
    _ = plt.plot(x, ((bounds[1] - bounds[0])/nbins)*(pdf), "-r", label="fit result")

In [ ]:
nbins = 80
plt.hist(data, bins=nbins, histtype='step', range=bounds);
plotfitresult(tot_model, bounds, nbins, data)
plt.xlabel("m [GeV/c$^2$]")
plt.ylabel("number of events")
plt.savefig("fit_discovery_ex.png")

### Discovery test

In a discovery test the null hypothesis is the absence of signal, .i.e Nsig = 0.

In [ ]:
# instantation of the calculator
calculator = AsymptoticCalculator(nll, minimizer)
calculator.bestfit = minimum #optionnal

In [ ]:
# parameter of interest of the null hypothesis
poinull = POI(Nsig, 0)

In [ ]:
# instantation of the discovery test
discovery_test = Discovery(calculator, [poinull])

In [ ]:
pnull, significance = discovery_test.result()